In [ ]:
from dsnp_qc_box import QcFunction 
import pandas as pd
import numpy as np
import datetime as dt
import os
from snp_query_box import DsnpHelperFunction
from dateutil.relativedelta import relativedelta

In [ ]:
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)

last_month = last_date_of_last_month_str[:7]
one_year_rolling_back_date = pd.Timestamp(last_date_of_last_month + dt.timedelta(days=1) - relativedelta(years = 1))
print("one_year_rolling_back_date")
print(one_year_rolling_back_date)

first_date_of_year = DsnpHelperFunction.first_date_of_year(today_str, output_type="string")
print("first_date_of_year")
print(first_date_of_year)
first_date_of_last_month_str = DsnpHelperFunction.first_date_of_last_month(today_str, output_type="string")
print("first_date_of_last_month")
print(first_date_of_last_month_str)

reporting_end_date = last_date_of_last_month_str
print("reporting_end_date")
print(reporting_end_date)

In [ ]:
last_reporting_end_date = DsnpHelperFunction.last_date_of_last_month(reporting_end_date, output_type="string")
last_reporting_end_date

In [ ]:
reporting_end_date

In [ ]:
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
flat_file_path = f'{storage_path}/{reporting_end_date}'
prev_flat_file_path = f'{storage_path}/{last_reporting_end_date}'

In [ ]:
#file_list = os.listdir(flat_file_path)
#file_list

In [ ]:
qc_file_list = [
#'pull_race.parquet',
#'pull_resident_type.parquet',
#'pull_risk_strat_from_icp.pkl',
#'pull_risk_strat_nj.pkl',
#'pull_risk_strat_va.pkl',
#'pull_utilization_12month.parquet',
 'qnxt_call_df.parquet',
 'not_real_df.pkl',
 'race_df.parquet',
 'sop_campaign_unique_df.parquet',
 'utilization_df_12month.parquet',
 'utilization_df_ytd.parquet']

In [ ]:
new_test_df = pd.read_pickle(f'{flat_file_path}/pull_risk_strat_va.pkl')

In [ ]:
new_test_df

In [ ]:
#test_df = pd.read_parquet(f'{prev_flat_file_path}/pull_annual_wellness_visit.parquet')
new_test_df = pd.read_parquet(f'{flat_file_path}/pull_risk_strat_from_icp.parquet')

In [ ]:
new_test_df

In [ ]:
new_test_df.VoicePermInd.value_counts()

In [ ]:
test_df

In [ ]:
0.439 + (0.439 * 0.2) 

In [ ]:
test_df.non_successful_rate.mean()

In [ ]:
new_test_df = pd.read_parquet(f'{flat_file_path}/med_adherence_df_with_detail.parquet')

In [ ]:
new_test_df.head()

In [ ]:
new_test_df.STATIN_DAYS_SUPPLY_CNT.max()

In [ ]:
new_test_df["DENOMINATOR_Breast Cancer Screening"].sum()

In [ ]:
647820.0 + (647820.0*0.15)

In [ ]:
test_df.head()

In [ ]:
test_df.mcl_legal_type_desc.value_counts()

In [ ]:
a = QcFunction.compare_dfs(test_df, test_df, check=["column"])
a

In [ ]:
a["column"]

In [ ]:
import flatfileQcHelper
import importlib
importlib.reload(flatfileQcHelper)

In [ ]:
import io
import sys
output_capture = io.StringIO()
sys.stdout = output_capture

for f in qc_file_list:
    file_name, file_type = f.split(".")
    print(f"{file_name}, QC start")
    if file_type == "parquet":
        try:
            prev_file = pd.read_parquet(f'{prev_flat_file_path}/{f}')
            new_file = pd.read_parquet(f'{flat_file_path}/{f}')
        except:
            print(f'{f} not found')
            print("-------------------------------------------------------------------")
            continue
    if file_type == "pkl":
        try:
            prev_file = pd.read_pickle(f'{prev_flat_file_path}/{f}')
            new_file = pd.read_pickle(f'{flat_file_path}/{f}')
        except:
            print(f'{f} not found')
            print("-------------------------------------------------------------------")
            continue
    
    print("Two dfs columns is the same?")
        
    print(QcFunction.compare_dfs(prev_file, new_file, check=["column"]))
    
    agg_dict = getattr(flatfileQcHelper, file_name + "_dict")
    agg_criteria = getattr(flatfileQcHelper, file_name + "_criteria")
    qc_dict, prev_agg, new_agg = QcFunction.compare_previous_report(prev_file, new_file, agg_dict, agg_criteria)
    qc_result = QcFunction.qc_result_from_qc_dict(qc_dict)
    print("-------------------------------------------------------------------")
    if qc_result == "Fail":
        QcFunction.send_email_alarm(report_name = f"Flatfile QC failed, {file_name}", dist_list = 'LimS@aetna.com;')

sys.stdout=sys.__stdout__
captured_output = output_capture.getvalue()

In [ ]:
captured_output_new_line = captured_output.replace('\n', '<br>')

In [ ]:
import os
import win32com.client as win32

# Construct outlook Application instance
olApp = win32.Dispatch('Outlook.Application')
olNS=olApp.GetNameSpace('MAPI')
#QC
CC_List = 'lims@aetna.com'
Distribution_List = 'lims@aetna.com;'

# Construct the email object
mailItem = olApp.CreateItem(0)
mailItem.Subject ='Monthly flatfile QC report'+ dt.datetime.today().strftime("%B %Y")
mailItem.BodyFormat = 1 ##'olFormatHTML'
mailItem.HTMLBody = f"{captured_output_new_line}"

mailItem.To = Distribution_List
mailItem.CC = CC_List
mailItem.Display()
mailItem.Save()
mailItem.Send()

In [ ]:
captured_output_new_line

In [ ]:
member_cm_engagement_df = pd.read_pickle(f'{flat_file_path}/member_cm_engagement_df.pkl')

In [ ]:
pbg_group_clean_df = pd.read_pickle(f'{flat_file_path}/pbg_group_clean_df.pkl')
hedis_ps_wide = pd.read_pickle(f'{flat_file_path}/hedis_ps_wide.pkl')

In [ ]:
import flatfileQcHelper
import importlib
importlib.reload(flatfileQcHelper)

hedis_qc_dict, hedis_aggregated_df = flatfileQcHelper.run_qc_hedis(hedis_ps_wide)
flatfileQcHelper.qc_alarm(hedis_qc_dict, report_name = "FlatFile QC", dist_list = "LimS@aetna.com")

pbg_group_qc_dict, pbg_group_aggregated_df = flatfileQcHelper.run_qc_pgb_group(pbg_group_clean_df)
flatfileQcHelper.qc_alarm(pbg_group_qc_dict, report_name = "FlatFile QC", dist_list = "LimS@aetna.com")

cm_qc_dict, cm_aggregated_df = flatfileQcHelper.run_qc_cm(member_cm_engagement_df)
flatfileQcHelper.qc_alarm(cm_qc_dict, report_name = "FlatFile QC", dist_list = "LimS@aetna.com")

In [ ]:
cm_aggregated_df